<a href="https://colab.research.google.com/github/plaban1981/NLP-with-Python/blob/master/NLP_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
dataframe = pd.read_csv('/content/drive/My Drive/NLP Tweets/train.csv')
test = pd.read_csv('/content/drive/My Drive/NLP Tweets/test.csv')

In [ ]:
test['target'] = -1

In [ ]:
data = pd.concat([dataframe,test])

In [ ]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
data.shape

(10876, 5)

In [ ]:
data.isnull().sum()

id             0
keyword       87
location    3638
text           0
target         0
dtype: int64

In [ ]:
data[['text','target']].head(10)

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
5,#RockyFire Update => California Hwy. 20 closed...,1
6,#flood #disaster Heavy rain causes flash flood...,1
7,I'm on top of the hill and I can see a fire in...,1
8,There's an emergency evacuation happening now ...,1
9,I'm afraid that the tornado is coming to our a...,1


In [ ]:
data['text'] = data['text'].apply(lambda x:str.lower(x))

In [ ]:
data['text']

0       our deeds are the reason of this #earthquake m...
1                  forest fire near la ronge sask. canada
2       all residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       just got sent this photo from ruby #alaska as ...
                              ...                        
3258    earthquake safety los angeles ûò safety faste...
3259    storm in ri worse than last hurricane. my city...
3260    green line derailment in chicago http://t.co/u...
3261    meg issues hazardous weather outlook (hwo) htt...
3262    #cityofcalgary has activated its municipal eme...
Name: text, Length: 10876, dtype: object

In [ ]:
#StopWords removal function
def remove_stopwords(string):
  word_list = nltk.tokenize.word_tokenize(string)
  stop = list(nltk.corpus.stopwords.words('english'))
  refine = [word for word in word_list if word not in stop]
  return ' '.join(refine)

In [ ]:
data['pure_text'] = data['text'].apply(remove_stopwords)

In [ ]:
# Removal of punctuation
def remove_punct(s):
  return ''.join([c for c in s if c not in string.punctuation])

data['ultra_pure_text'] = data['pure_text'].apply(remove_punct)

In [ ]:
import re

def remove_URL(text):
  url = re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'',text)

def remove_numbers(text):
  number = re.compile(r'\d')
  return number.sub(r'',text)

In [ ]:
data['ultra_pure_text'] = data['ultra_pure_text'].apply(remove_URL)
data['ultra_pure_text'] = data['ultra_pure_text'].apply(remove_numbers)

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download("wordnet")
nltk.download('punkt')
def lemma(text):
  words = nltk.word_tokenize(text)
  sen = [wordnet_lemmatizer.lemmatize(i, pos="v") for i in words]
  return " ".join(sen)


data['ultra_pure_text'] = data['ultra_pure_text'].apply(lemma)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
dataframe['target'].value_counts()[0]/7613

0.5703402075397347

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
data['len_tweets'] = data['ultra_pure_text'].apply(len)

In [ ]:
data['len_tweets'].describe()

count    10876.000000
mean        72.860886
std         26.606034
min          0.000000
25%         55.000000
50%         76.000000
75%         94.000000
max        137.000000
Name: len_tweets, dtype: float64

In [ ]:
# So we will consider first 80 characters of the tweets
# Best config -> max_words = 10000 maxlen = 100


maxlen = 100
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['ultra_pure_text'])
sequences = tokenizer.texts_to_sequences(data['ultra_pure_text'])

word_index = tokenizer.word_index

In [ ]:
data1 = pad_sequences(sequences,maxlen=maxlen)

In [ ]:
len(word_index) # Total this many words found!

25493

In [ ]:
data1.shape

(10876, 100)

In [ ]:
type(data1)

numpy.ndarray

In [ ]:
embeddings_index = {}
f = open('/content/drive/My Drive/NLP Tweets/glove.6B.300d.txt')

for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:],dtype='float32')
  embeddings_index[word] = coefs

f.close()

In [ ]:
from keras.utils import to_categorical

train = data1[:6613,:]
y_train = dataframe['target'][:6613]
#y_train = to_categorical(y_train)

val = data1[6613:7613,:]
y_val = dataframe['target'][6613:7613]
#y_val = to_categorical(y_val)

xtest = data1[7613:,:]

In [ ]:
embedding_dim = 300

embedding_matrix = np.zeros((max_words,embedding_dim))

for word,i in word_index.items():
  if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense,Dropout,Conv1D,MaxPool1D,LSTM,SimpleRNN,BatchNormalization
from keras import regularizers

In [ ]:
"""
BEST MODEL


model = Sequential()

model.add(Embedding(max_words,embedding_dim,input_length=maxlen))

model.add(BatchNormalization(axis=1))


model.add(Conv1D(64,kernel_size=(2),activation='relu',kernel_regularizer=regularizers.l1(0.0006)))
model.add(MaxPool1D(2))

model.add(Conv1D(32,kernel_size=(2),activation='relu',kernel_regularizer=regularizers.l1(0.0001)))
model.add(MaxPool1D(2))


model.add(Flatten())
model.add(Dropout(0.25))


model.add(Dense(16,activation='relu',kernel_regularizer=regularizers.l2(0.07)))

model.add(Dense(1,activation='sigmoid'))



"""

"\nBEST MODEL\n\n\nmodel = Sequential()\n\nmodel.add(Embedding(max_words,embedding_dim,input_length=maxlen))\n\nmodel.add(BatchNormalization(axis=1))\n\n\nmodel.add(Conv1D(64,kernel_size=(2),activation='relu',kernel_regularizer=regularizers.l1(0.0006)))\nmodel.add(MaxPool1D(2))\n\nmodel.add(Conv1D(32,kernel_size=(2),activation='relu',kernel_regularizer=regularizers.l1(0.0001)))\nmodel.add(MaxPool1D(2))\n\n\nmodel.add(Flatten())\nmodel.add(Dropout(0.25))\n\n\nmodel.add(Dense(16,activation='relu',kernel_regularizer=regularizers.l2(0.07)))\n\nmodel.add(Dense(1,activation='sigmoid'))\n\n\n\n"

In [ ]:
model = Sequential()

model.add(Embedding(max_words,embedding_dim,input_length=maxlen))

model.add(BatchNormalization(axis=1))


model.add(Conv1D(64,kernel_size=(2),activation='relu',kernel_regularizer=regularizers.l1(0.0006)))
model.add(MaxPool1D(2))

model.add(Conv1D(32,kernel_size=(2),activation='relu',kernel_regularizer=regularizers.l1(0.0001)))
model.add(MaxPool1D(2))


model.add(Flatten())
model.add(Dropout(0.25))


model.add(Dense(16,activation='relu',kernel_regularizer=regularizers.l2(0.07)))

model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 300)          3000000   
_________________________________________________________________
batch_normalization_7 (Batch (None, 100, 300)          400       
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 99, 64)            38464     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 49, 64)            0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 48, 32)            4128      
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 24, 32)            0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 768)              

In [ ]:
train.shape
from keras.optimizers import Adam

In [ ]:
model.compile(optimizer=Adam(lr=0.0001),loss=root_mean_squared_error,metrics=['accuracy'])
model.fit(train,y_train,batch_size=128,epochs=30)

NameError: ignored

In [ ]:
results = model.evaluate(val,y_val)

1000/1000 [==============================] - 0s 315us/step


In [ ]:
results

[0.8890468006134034, 0.805]

In [ ]:
preds = model.predict(xtest)

In [ ]:
preds

array([[0.5164512 ],
       [0.85088354],
       [0.93354976],
       ...,
       [0.3751279 ],
       [0.89215285],
       [0.43308294]], dtype=float32)

In [ ]:
# y_pred1=[]

# for i in preds:
#   if i[0]>0.5:
#     y_pred1.append(0)
#   else:
#      y_pred1.append(1)

# y_pred1=np.array(y_pred1)

In [ ]:
result = []
for pred in preds:
  if pred > 0.5:
    pred = 1
  else:
    pred = 0
  result.append(pred)

In [ ]:
test['target'] = result

In [ ]:
test[['id','target']].to_csv('/content/drive/My Drive/NLP Tweets/nlp_result.csv',index=False)

DENSE LAYER ONLY

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(data['ultra_pure_text']).toarray()

In [ ]:
X.shape

(10876, 3000)

In [ ]:
train = X[:6613,:]
y_train = dataframe['target'][:6613]

val = X[6613:7613,:]
y_val = dataframe['target'][6613:7613]

xtest = X[7613:,:]

In [ ]:
train.shape

(6613, 3000)

In [ ]:
model1 = Sequential()
model1.add(Dense(256,activation='relu',input_shape=(3000,)))
#model1.add(Dropout(0.25))
#model1.add(Dense(128,activation='relu'))
#model1.add(Dense(256,activation='relu'))
#model1.add(Dropout(0.15))
model1.add(Dense(64,activation='relu'))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(optimizer=Adam(lr=0.0001),loss='binary_crossentropy',metrics=['accuracy'])
model1.fit(train,y_train,batch_size=64,epochs=20,class_weight='balanced')

Epoch 1/20
6613/6613 [==============================] - 2s 285us/step - loss: 0.5585 - acc: 0.7122
Epoch 2/20
6613/6613 [==============================] - 1s 111us/step - loss: 0.3490 - acc: 0.8533
Epoch 3/20
6613/6613 [==============================] - 1s 116us/step - loss: 0.2559 - acc: 0.9029
Epoch 4/20
6613/6613 [==============================] - 1s 112us/step - loss: 0.1832 - acc: 0.9345
Epoch 5/20
6613/6613 [==============================] - 1s 114us/step - loss: 0.1326 - acc: 0.9545
Epoch 6/20
6613/6613 [==============================] - 1s 111us/step - loss: 0.0969 - acc: 0.9663
Epoch 7/20
6613/6613 [==============================] - 1s 115us/step - loss: 0.0803 - acc: 0.9699
Epoch 8/20
6613/6613 [==============================] - 1s 113us/step - loss: 0.0683 - acc: 0.9722
Epoch 9/20
6613/6613 [==============================] - 1s 114us/step - loss: 0.0620 - acc: 0.9754
Epoch 10/20
6613/6613 [==============================] - 1s 111us/step - loss: 0.0601 - acc: 0.9744
Epoch 11/

In [ ]:
model1.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 256)               768256    
_________________________________________________________________
dense_58 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 65        
Total params: 784,769
Trainable params: 784,769
Non-trainable params: 0
_________________________________________________________________


In [ ]:
results = model1.evaluate(val,y_val)

1000/1000 [==============================] - 1s 548us/step


In [ ]:
results

[1.5060250771045685, 0.717]

In [ ]:
preds = model.predict(xtest)

In [ ]:
result = []
for pred in preds:
  if pred > 0.5:
    pred = 1
  else:
    pred = 0
  result.append(pred)

In [ ]:
test['target'] = result

In [ ]:
test[['id','target']].to_csv('/content/drive/My Drive/NLP Tweets/nlp_result.csv',index=False)

In [ ]:
dataframe['target'][6613:].shape

(1000,)